<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-research/perch-hoplite/blob/main/perch_hoplite/agile/1_embed_audio_v2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-research/perch-hoplite/blob/main/perch_hoplite/agile/1_embed_audio_v2.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Overview

This notebook uses the latest version, 0.1.3 of the github repo, and thus allowing poetry to install perch-hoplite==1.0.0.dev. Whereas repo 0.1.1 has poetry install perch-hoplite==0.1.1

this notebook uses my setup, which uses poetry. It combines both the embeddings and the agile modelling notebooks I pull from the github repo

In [ ]:
!git clone https://github.com/google-research/perch-hoplite.git

In [ ]:
%cd perch-hoplite
!pip install poetry
!poetry config virtualenvs.create false

In [ ]:
!poetry install

In [ ]:
!nvidia-smi

In [ ]:
# @title Imports

from etils import epath
from IPython.display import display
import ipywidgets as widgets
from ml_collections import config_dict
import numpy as np

from perch_hoplite.agile import colab_utils
from perch_hoplite.agile import embed
from perch_hoplite.agile import source_info
from perch_hoplite.db import brutalism
from perch_hoplite.db import interface
from perch_hoplite.zoo import taxonomy_model_tf

# Embed the audio data

Mount our google drive, and then load the .wav files into the config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!ls

In [ ]:
# @title Configuration {vertical-output: true}

# @markdown Configure the raw dataset and output location(s).  The format is a mapping from
# @markdown a dataset_name to a (base_path, fileglob) pair.  Note that the file
# @markdown globs are case sensitive.  The dataset name can be anything you want.
#
# @markdown This structure allows you to move your data around without having to
# @markdown re-embed the dataset.  The generated embedding database will be
# @markdown placed in the base path. This allows you to simply swap out
# @markdown the base path here if you ever move your dataset.

# @markdown By default we only process one dataset at a time.  Re-run this entire notebook
# @markdown once per dataset.

# @markdown For example, we might set dataset_base_path to `/home/me/myproject`,
# @markdown and use the glob `*/*.wav` if all of the audio files have filepaths
# @markdown like `/home/me/myproject/site_XYZ/audio_ABC.wav` (e.g. audio files are contained
# @markdown in subfolders of the base directory).

# @markdown 1. Create a unique name for the database that will store the embeddings for the
# @markdown target data.
dataset_name = 'deep_sea_data_raw2'  # @param {type: 'string'}

# @markdown 2. Input the filepath for the folder that is containing the input audio files.
dataset_base_path = '/content/drive/MyDrive/ns_experiment/deep_sea_data_raw2'  # @param {type: 'string'}

# @markdown 3. Input the file pattern for the audio files within that folder that you want
# @markdown to embed. Some examples for how to input:
# @markdown - All files in the base directory of a specific type (not subdirectories): e.g.
# @markdown `*.wav` (or `*.flac` etc) will generate embeddings for all .wav files (or whichever
# @markdown format) in the `dataset_base_path`.
# @markdown - All files in one level of subdirectories within the base directory: `*/*.flac`
# @markdown will generate embeddings for all .flac files.
# @markdown - Single file: `myfile.wav` will only embed the audio from that specific file.
dataset_fileglob = '*/*.wav'  # @param {type:'string'}

# @markdown 4. [Optional] If saving the embeddings database to a new directory, specify here.
# @markdown Otherwise, leave blank - by default the embeddings database output will be saved
# @markdown within `dataset_base_path` where the audio is located. You do not need to specify
# @markdown `db_path` unless you want to maintain multiple distinct embedding databases, or
# @markdown if you would like to save the output in a different folder. If your input audio
# @markdown data is accessed from a public URL, we recommend specifying a separate output
# @markdown directory here.
db_path = ''  # @param {type:'string'}
if not db_path or db_path.lower() == 'none':
  db_path = None

# @markdown 5. Choose a supported model to generate embeddings. `perch_v2` is the latest Perch
# @markdown model and was trained on multiple taxa include birds, mammals, insects and amphibians.
# @markdown `perch_v2` has also demonstrated high performance for marine audio transfer learning
# @markdown tasks. **NOTE: `perch_v2` only works with GPU runtimes - see above instructions.**
# @markdown `perch_8` is the last updated version of Perch V1 trained only on birds, and
# @markdown `birdnet_v2.3` is also a common option for birds. Other choices include `surfperch`
# @markdown for coral reefs or `multispecies_whale` for marine mammals.
model_choice = 'perch_v2'  # @param ['perch_v2', 'perch_8', 'humpback', 'multispecies_whale', 'surfperch', 'birdnet_V2.3']

# @markdown 6. [Optional] Shard the audio for embeddings. File sharding automatically splits audio
# @markdown files into smaller chunks for creating embeddings. This limits both system and GPU
# @markdown memory usage, especially useful when working with long files (>1 hour).
use_file_sharding = True  # @param {type:'boolean'}
# @markdown If you want to change the length in seconds for the shards, specify here.
shard_length_in_seconds = 60  # @param {type:'number'}

audio_glob = source_info.AudioSourceConfig(
    dataset_name=dataset_name,
    base_path=dataset_base_path,
    file_glob=dataset_fileglob,
    min_audio_len_s=1.0,
    target_sample_rate_hz=-2,
    shard_len_s=float(shard_length_in_seconds) if use_file_sharding else None,
)

configs = colab_utils.load_configs(
    source_info.AudioSources((audio_glob,)),
    db_path,
    model_config_key=model_choice,
    db_key='sqlite_usearch',
)
configs

In [ ]:
# @title Initialize the hoplite database (DB) {vertical-output: true}

global db
db = configs.db_config.load_db()
num_embeddings = db.count_embeddings()

print('Initialized DB located at:', configs.db_config.db_config.db_path)

def drop_and_reload_db(_) -> interface.HopliteDBInterface:
  db_path = epath.Path(configs.db_config.db_config.db_path)
  for fp in db_path.glob('hoplite.sqlite*'):
    fp.unlink()
  (db_path / 'usearch.index').unlink()
  print('\n Deleted previous db at: ', configs.db_config.db_config.db_path)
  db = configs.db_config.load_db()

# @markdown If `drop_existing_db` set to True, when the database already exists and contains
# @markdown embeddings, then those existing embeddings will be erased. You will be prompted
# @markdown to confirm you wish to delete those existing embeddings. If you want to keep
# @markdown existing embeddings in the database, then set to False, which will append the new
# @markdown embeddings to the database.
drop_existing_db = True  # @param {type: 'boolean'}

if num_embeddings > 0 and drop_existing_db:
  print('Existing DB contains projects: ', db.get_all_projects())
  print('num embeddings: ', num_embeddings)
  print('\n\nClick the button below to confirm you really want to drop the database at ')
  print(f'{configs.db_config.db_config.db_path}\n')
  print(f'This will permanently delete all {num_embeddings} embeddings from the existing database.\n')
  print('If you do NOT want to delete this data, set `drop_existing_db` above to `False` and re-run this cell.\n')

  button = widgets.Button(description='Delete database?')
  button.on_click(drop_and_reload_db)
  display(button)

In [ ]:
# @title Run the embedding {vertical-output: true}

print(f'Embedding dataset as a new db project: {audio_glob.dataset_name}')

worker = embed.EmbedWorker(
    audio_sources=configs.audio_sources_config,
    db=db,
    model_config=configs.model_config,
)

worker.process_all(target_dataset_name=audio_glob.dataset_name)

print('\n\nEmbedding complete, total embeddings: ', db.count_embeddings())

In [ ]:
# @title Per project statistics {vertical-output: true}

for project in db.get_all_projects():
  window_ids = db.match_window_ids(
      deployments_filter=config_dict.create(eq=dict(project=project))
  )
  print('Project:', project)
  print('>>> num embeddings:', len(window_ids))
  print()

In [ ]:
# @title Show example embedding search
# @markdown As an example (and to show that the embedding process worked), this selects a single
# @markdown embedding from the database and outputs the embedding ids of the top-k (k = 128)
# @markdown nearest neighbors in the database.

q = db.get_embedding(db.match_window_ids(limit=1)[0])
%time results, scores = brutalism.brute_search(worker.db, query_embedding=q, search_list_size=128, score_fn=np.dot)
print([int(r.window_id) for r in results])

Here the agile modeling starts

In [ ]:
# @title Imports

import os

from matplotlib import pyplot as plt
import numpy as np

from perch_hoplite.agile import audio_loader
from perch_hoplite.agile import classifier
from perch_hoplite.agile import classifier_data
from perch_hoplite.agile import embedding_display
from perch_hoplite.agile import source_info
from perch_hoplite.db  import brutalism
from perch_hoplite.db import score_functions
from perch_hoplite.db  import search_results
from perch_hoplite.db import sqlite_usearch_impl
from perch_hoplite.zoo import model_configs
from perch_hoplite.zoo import taxonomy_model_tf

In [ ]:
# @title Load model and connect to database {vertical-output: true}

# @markdown Location of database containing audio embeddings.
db_path = '/content/drive/MyDrive/ns_experiment/deep_sea_data_raw2'  # @param {type: 'string'}

# @markdown Identifier (e.g. name) to attach to labels produced during validation.
annotator_id = 'Mas'  # @param {type: 'string'}

db = sqlite_usearch_impl.SQLiteUSearchDB.create(db_path)
db_model_config = db.get_metadata('model_config')
embed_config = db.get_metadata('audio_sources')
model_class = model_configs.get_model_class(db_model_config.model_key)
embedding_model = model_class.from_config(db_model_config.model_config)
audio_sources = source_info.AudioSources.from_config_dict(embed_config)
if hasattr(embedding_model, 'window_size_s'):
  window_size_s = embedding_model.window_size_s
else:
  window_size_s = 5.0
audio_filepath_loader = audio_loader.make_filepath_loader(
    audio_sources=audio_sources,
    window_size_s=window_size_s,
    sample_rate_hz=embedding_model.sample_rate,
)

In [ ]:
# @title Load query audio {vertical-output: true}

# @markdown The `query_uri` can be a URL, filepath, or Xeno-Canto ID
# @markdown (like `xc1038305`, containing an Eastern Towhee (`eastow`)).
query_uri = '/content/drive/MyDrive/ns_experiment/deep_sea_data_raw2/ZOOM0003/clip_001695.wav'  # @param {type: 'string'}
query_label = 'fish'  # @param {type: 'string'}

query = embedding_display.QueryDisplay(
    uri=query_uri, offset_s=0.0, window_size_s=5.0, sample_rate_hz=32000)
_ = query.display_interactive()

In [ ]:
# @title Embed the Query and Search {vertical-output: true}

# @markdown Number of results to find and display.
num_results = 50  # @param
query_embedding = embedding_model.embed(
    query.get_audio_window()).embeddings[0, 0]

# @markdown If checked, search for examples near a particular target score.
target_sampling = False  # @param {type: 'boolean'}

# @markdown When target sampling, target this score.
target_score = -1.0  # @param
if not target_sampling:
  target_score = None

# @markdown If True, search the full DB. Otherwise, use approximate
# @markdown nearest-neighbor search. (Mas comment: i believe false is usefull
# @markdown for very large datasets, e.g. >100k embeds, check conclusion
# @markdown in paper stating recall might be lower?
exact_search = True  # @param {type: 'boolean'}

if exact_search:
  score_fn = score_functions.get_score_fn('dot', target_score=target_score)
  results, all_scores = brutalism.threaded_brute_search(
      db, query_embedding, num_results, score_fn=score_fn)
  if not target_sampling:
    # TODO(tomdenton): Create a reasonable histogram when target sampling.
    all_scores = [s for s in all_scores if s == target_score]
    _ = plt.hist(all_scores, bins=100)
    hit_scores = [r.sort_score for r in results.search_results]
    plt.scatter(hit_scores, np.zeros_like(hit_scores), marker='|',
                color='r', alpha=0.5)
else:
  ann_matches = db.ui.search(query_embedding, count=num_results)
  results = search_results.TopKSearchResults(top_k=num_results)
  for k, d in zip(ann_matches.keys, ann_matches.distances):
    results.update(search_results.SearchResult(k, d))


In [ ]:
# @title Display Results {vertical-output: true}

display_results = embedding_display.EmbeddingDisplayGroup.from_search_results(
    results, db, sample_rate_hz=32000, frame_rate=100,
    audio_loader=audio_filepath_loader)
display_results.display(positive_labels=[query_label])

In [ ]:
# @title Save data labels {vertical-output: true}

print("Annotations before saving new labels:", len(db.get_all_annotations()))

for ann in display_results.harvest_labels(annotator_id):
  db.insert_annotation(
      window_id=ann.window_id,
      label=ann.label,
      label_type=ann.label_type,
      provenance=ann.provenance,
      skip_duplicates=True,
  )

print("Annotations after saving new labels:", len(db.get_all_annotations()))

In [ ]:
# @title Classifier training {vertical-output: true}

# @markdown Set of labels to classify. If None, auto-populated from the DB.
target_labels = None  # @param

# @markdown Classifier traning hyperparams. These should not require tuning.
learning_rate = 1e-3  # @param
weak_neg_weight = 0.05  # @param
l2_mu = 0.000  # @param
num_steps = 128  # @param

train_ratio = 0.9  # @param
batch_size = 128  # @param
weak_negatives_batch_size = 128  # @param
loss_fn_name = 'bce'  # @param ['hinge', 'bce']

data_manager = classifier_data.AgileDataManager(
    target_labels=target_labels,
    db=db,
    train_ratio=train_ratio,
    min_eval_examples=1,
    batch_size=batch_size,
    weak_negatives_batch_size=weak_negatives_batch_size,
    rng=np.random.default_rng(seed=5))
print('Training for target labels : ')
print(data_manager.get_target_labels())
linear_classifier, eval_scores = classifier.train_linear_classifier(
    data_manager=data_manager,
    learning_rate=learning_rate,
    weak_neg_weight=weak_neg_weight,
    num_train_steps=num_steps,
)
print('\n' + '-' * 80)
top1 = eval_scores['top1_acc']
print(f'top-1      {top1:.3f}')
rocauc = eval_scores['roc_auc']
print(f'roc_auc    {rocauc:.3f}')
cmap = eval_scores['cmap']
print(f'cmap       {cmap:.3f}')

# Save linear classifier.
linear_classifier.save(os.path.join(db_path, 'agile_classifier_v2.pt'))

In [ ]:
# @title Review Classifier Results {vertical-output: true}

# @markdown Number of results to find and display.
target_label = 'fish'  # @param {type: 'string'}
num_results = 50  # @param

target_label_idx = data_manager.get_target_labels().index(target_label)
class_query = linear_classifier.beta[:, target_label_idx]
bias = linear_classifier.beta_bias[target_label_idx]

# @markdown Number of (randomly selected) database entries to search over.
sample_size = 1_000_000  # @param

# @markdown Whether to use margin-sampling. If checked, search for examples
# @markdown with logits near a particular target score (usually 0).
margin_sampling = False  # @param {type: 'boolean'}

# @markdown When margin sampling, target this logit.
margin_target_score = -0.0  # @param
if not margin_sampling:
  margin_target_score = None
score_fn = score_functions.get_score_fn(
    'dot', bias=bias, target_score=margin_target_score)
results, all_scores = brutalism.threaded_brute_search(
    db, class_query, num_results, score_fn=score_fn,
    sample_size=sample_size)

if not margin_sampling:
  # TODO(tomdenton): Create a reasonable histogram when margin sampling.
  _ = plt.hist(all_scores, bins=100)
  hit_scores = [r.sort_score for r in results.search_results]
  plt.scatter(hit_scores, np.zeros_like(hit_scores), marker='|',
              color='r', alpha=0.5)


In [ ]:
# @title Display Results {vertical-output: true}

display_results = embedding_display.EmbeddingDisplayGroup.from_search_results(
    results, db, sample_rate_hz=32000, frame_rate=100,
    audio_loader=audio_filepath_loader)
display_results.display(positive_labels=[target_label])

In [ ]:
# @title Save data labels {vertical-output: true}

print("Annotations before saving new labels:", len(db.get_all_annotations()))

for ann in display_results.harvest_labels(annotator_id):
  db.insert_annotation(
      window_id=ann.window_id,
      label=ann.label,
      label_type=ann.label_type,
      provenance=ann.provenance,
      skip_duplicates=True,
  )

print("Annotations after saving new labels:", len(db.get_all_annotations()))

In [ ]:
# @title Run inference with trained classifier {vertical-output: true}

output_csv_filepath = '/content/drive/MyDrive/ns_experiment/deep_sea_data_raw2/inference.csv'  # @param {type: 'string'}
logit_threshold = 1.0  # @param
# Set labels to a tuple of desired labels if you want to run inference on
# a subset of the labels.
labels = None  # @param

classifier.write_inference_csv(
    linear_classifier,
    db,
    output_csv_filepath,
    logit_threshold,
    labels=labels,
    window_ids=db.match_window_ids(),
)